In [2]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *
from scapy.layers.inet import IP,TCP
import requests
from random import randint
import pickle
from concurrent.futures import ThreadPoolExecutor

In [12]:
def get_tcp_opt(p, opt, default):
    if opt in map(lambda tup : tup[0], p['TCP'].options):
        return reduce(lambda opt1,opt2 : opt1 if opt1[0]==opt else opt2, p['TCP'].options)[1]
    return default

def send_http_req(ip):
    try:
        tmp = requests.get(f'https://{ip}')
    except:
        pass

In [22]:
ips, features = [], {}

sniffer = AsyncSniffer(filter='src port 443 and tcp[tcpflags] & (tcp-syn|tcp-ack|tcp-push) == (tcp-syn|tcp-ack)')
sniffer.start()

src_dir = 'C:\\Users\\ofir\\Downloads\\data\\ips'
for filename in sorted(os.listdir(src_dir))[:5]:
    print(f'working on {filename}...')
    with open('%s\\%s' % (src_dir, filename), 'rt') as curr_input_file:
        ips = list(map(lambda line : line.rstrip('\n'), curr_input_file))
    
    with ThreadPoolExecutor(max_workers = 100) as executor:
        executor.map(send_http_req, ips)

res = sniffer.stop()

In [24]:
for SA in res:
    curr_data = {
        'df'      : 1 if 'DF' in str(SA['IP'].flags) else 0 ,
        'ttl'     : SA['IP'].ttl ,
        'w_size'  : SA['TCP'].window ,
        'mss'     : get_tcp_opt(SA, 'MSS', default=-1) ,
        'w_scale' : get_tcp_opt(SA, 'WScale', default=1) ,

        'df+'  : 1  if (    'DF' in str(SA['IP'].flags) and not SA['IP'].id)    else 0 ,
        'df-'  : 1  if (not 'DF' in str(SA['IP'].flags) and     SA['IP'].id)    else 0 ,
        'fo+'  : 1  if (not 'DF' in str(SA['IP'].flags) and     SA['IP'].frag)  else 0 ,
        'fo-'  : 1  if (    'DF' in str(SA['IP'].flags) and not SA['IP'].frag)  else 0 ,

        'ecn'  : 1  if ('E' in SA['TCP'].flags)                                else 0 ,
        'seq0' : 1  if (SA['TCP'].seq == 0)                                    else 0 ,

        'opts' : ','.join([opt[0] for opt in SA['TCP'].options])
    }
    features[SA['IP'].src] = curr_data

In [25]:
# for p in res:
#     print(f"src:{p['IP'].src}:{p['TCP'].sport}\n\t{p['TCP'].options}")
import pandas as pd
# df = pd.DataFrame.from_dict(features.values())
# df.describe()

df = pd.DataFrame(map(lambda val : val['opts'], list(features.values())))
df.value_counts()

MSS,NOP,NOP,SAckOK,NOP,WScale    399
MSS,NOP,WScale,NOP,NOP,SAckOK     32
MSS,NOP,WScale,SAckOK,EOL         16
MSS,SAckOK,EOL                    13
MSS,NOP,WScale                     7
MSS                                1
MSS,NOP,NOP,SAckOK                 1
MSS,WScale,NOP,SAckOK,NOP,NOP      1
dtype: int64